In [ ]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# importing the dataset
anime_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/anime-recommender/dataset/anime.csv')
rating_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/anime-recommender/dataset/rating.csv')
print(anime_df.shape)
print(rating_df.shape)

(12294, 7)
(7813737, 3)


In [ ]:
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


# General or Non-Personal Recommendations

This type of recommendation system does not require the personal details of the user. It just recommends the best anime based on the collective ratings of the entire fanbase.

In [ ]:
# Generalized or Impersonal Recommendations 
anime_df['Weighted Rating'] = (anime_df['rating']*anime_df['members'])
anime_df.sort_values(by='Weighted Rating',ascending=False)[['anime_id','name','genre','type','rating']].head(50)

,anime_id,name,genre,type,rating
40,1535,Death Note,"Mystery, Police, Psychological, Supernatural, ...",TV,8.71
86,16498,Shingeki no Kyojin,"Action, Drama, Fantasy, Shounen, Super Power",TV,8.54
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,9.26
804,11757,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",TV,7.83
19,1575,Code Geass: Hangyaku no Lelouch,"Action, Mecha, Military, School, Sci-Fi, Super...",TV,8.83
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,9.17
159,6547,Angel Beats!,"Action, Comedy, Drama, School, Supernatural",TV,8.39
131,4224,Toradora!,"Comedy, Romance, School, Slice of Life",TV,8.45
841,20,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,7.81
445,10620,Mirai Nikki (TV),"Action, Mystery, Psychological, Shounen, Super...",TV,8.07


# Collaborative Filtering (Item-Item nearest Neighbor)

The algorithm aims to find the most similar items to the given item nd it recommends those items to the user who has given good rating to the given item.

In [ ]:
# Collaborative Filtering
pv_table = rating_df[rating_df['rating'] != -1].drop_duplicates(subset=['user_id','anime_id']).pivot(index='user_id',columns='anime_id',values='rating')
pv_table

anime_id,1,5,6,7,8,15,16,17,18,19,20,22,23,24,25,26,27,28,29,30,31,32,33,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,...,33862,33866,33868,33869,33870,33871,33872,33885,33902,33905,33906,33907,33908,33909,33910,33911,33912,33914,33915,33934,33964,33979,34015,34036,34048,34085,34103,34107,34119,34136,34238,34239,34240,34252,34283,34324,34325,34349,34367,34475
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,8.0,NaN,NaN,6.0,NaN,6.0,6.0,NaN,6.0,5.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73513,9.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# get anime name from user
anime_name = input('Enter anime name: ')
anime_id = int(anime_df[anime_df['name'] == anime_name]['anime_id'])

# finding movies similar to mentioned anime
selected_anime = pv_table[anime_id]
sim_to_anime = pv_table.corrwith(selected_anime)
selected_anime_corrwith = pd.DataFrame({ "Anime":pv_table.columns,"Correlation" :sim_to_anime})
sim_to_anime = selected_anime_corrwith.dropna().sort_values(by='Correlation',ascending=False).reset_index()[['Anime','Correlation']]
sim_to_anime.head()

Enter anime name: Hunter x Hunter (2011)


/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,Anime,Correlation
0,1758,1.0
1,5272,1.0
2,31695,1.0
3,3396,1.0
4,28815,1.0


As we can see, some animes have perfect positive correlation with the given anime. This is not possible unless any one of the users who watched a popular anime also watched an unpopular anime which was not rated by any other user. In order to remove this problem, we want the number of ratings given to the unpopular movie to be greater than a particular threshold. This value is to be played with.

In [ ]:
# removing inconsistensies
anime_number_df = rating_df.groupby('anime_id')[['rating']].sum()
sim_to_anime = sim_to_anime.merge(anime_number_df.reset_index().rename(columns={'anime_id':'Anime','rating':'No of Ratings'}),on='Anime')
sim_to_anime[sim_to_anime['No of Ratings'] > 200]

,Anime,Correlation,No of Ratings
47,4154,1.000000,236
84,1411,1.000000,222
122,11061,1.000000,67949
183,2289,0.997176,252
188,2484,0.983135,206
...,...,...,...
7323,4375,-1.000000,228
7329,1900,-1.000000,321
7367,3305,-1.000000,223
7436,2200,-1.000000,209


In [ ]:
sim_to_anime[(sim_to_anime['Correlation'] <= 0.90) &  (sim_to_anime['No of Ratings'] >= 300)].merge(anime_df.rename(columns={'anime_id':'Anime'})[['Anime','name']],on='Anime').iloc[:10]

,Anime,Correlation,No of Ratings,name
0,1755,0.894427,330,Di Gi Charat Ohanami Special
1,1207,0.880705,306,Wata no Kuni Hoshi
2,2060,0.876356,543,Gall Force 1: Eternal Story
3,708,0.850634,405,Patapata Hikousen no Bouken
4,349,0.818881,416,Magical Canan
5,2570,0.816161,312,Ochame na Futago: Claire Gakuin Monogatari
6,2419,0.813489,389,Spirit of Wonder: China-san no Yuuutsu
7,1977,0.810130,353,Hijikata Toshizou: Shiro no Kiseki
8,5165,0.800853,397,Macross 25th Anniversary: All That VF Macross ...
9,1832,0.795181,499,Meiou Project Zeorymer


Finally , Recommendations are made based on the anime that correlate best with the given anime.